# Load data

In [1]:
# Save for running other methods
import numpy as np
d1 = np.load('../plots/dyngen/dyngen_logcounts.npy')
d2 = np.load('../plots/dyngen/dyngen_protein.npy')
m1 = np.load('../plots/dyngen/dyngen_logcounts_pre.npy')
m2 = np.load('../plots/dyngen/dyngen_protein_pre.npy')
mask = np.load('../plots/dyngen/dyngen_mask.npy')

# PCA

In [ ]:
# Apply PCA
import sklearn.decomposition
method = 'PCA'
def apply_red(x):
    red = sklearn.decomposition.PCA(n_components=32, random_state=42)
    red.fit(x[mask])
    return red.inverse_transform(red.transform(x))
np.save(f'../plots/dyngen/{method}_logcounts.npy', apply_red(d1))
np.save(f'../plots/dyngen/{method}_protein.npy', apply_red(d2))
np.save(f'../plots/dyngen/{method}_logcounts_pre.npy', apply_red(m1))
np.save(f'../plots/dyngen/{method}_protein_pre.npy', apply_red(m2))

# Autoencoder

In [ ]:
# Apply autoencoder
import sklearn.neural_network
method = 'Autoencoder'
def apply_red(x):
    red = sklearn.neural_network.MLPRegressor(
        hidden_layer_sizes=(2*x.shape[1], 32, 2*x.shape[1]),
        # verbose=True,
        random_state=42)
    red.fit(x[mask], x[mask])
    return red.predict(x)
np.save(f'../plots/dyngen/{method}_logcounts.npy', apply_red(d1))
np.save(f'../plots/dyngen/{method}_protein.npy', apply_red(d2))
np.save(f'../plots/dyngen/{method}_logcounts_pre.npy', apply_red(m1))
np.save(f'../plots/dyngen/{method}_protein_pre.npy', apply_red(m2))

# UMAP

In [ ]:
# import umap
# method = 'UMAP'
# def apply_red(x):
#     red = umap.UMAP(n_components=32, random_state=42)
#     lat = red.fit_transform(x)
#     return red.inverse_transform(lat)
# np.save(f'../plots/dyngen/{method}_logcounts.npy', apply_red(d1))
# np.save(f'../plots/dyngen/{method}_protein.npy', apply_red(d2))
# np.save(f'../plots/dyngen/{method}_logcounts_pre.npy', apply_red(m1))
# np.save(f'../plots/dyngen/{method}_protein_pre.npy', apply_red(m2))

# JAMIE

In [ ]:
import torch
import jamie  # Needs to be installed separately
method = 'JAMIE'
def apply_red(x1, x2):
    np.random.seed(42); torch.manual_seed(42)
    np.random.seed(42); torch.manual_seed(42)
    red = jamie.JAMIE(output_dim=32)
    red.fit_transform([x1, x2])
    return [
        red.model.preprocessing_inverse[0](red.model.impute(torch.tensor(red.model.preprocessing[0](d1)).float(), compose=[0, 0]).cpu().detach()),
        red.model.preprocessing_inverse[1](red.model.impute(torch.tensor(red.model.preprocessing[1](d2)).float(), compose=[1, 1]).cpu().detach())]
out = apply_red(d1, d2)
np.save(f'../plots/dyngen/{method}_logcounts.npy', out[0])
np.save(f'../plots/dyngen/{method}_protein.npy', out[1])
# out = apply_red(m1, m2)
# np.save(f'../plots/dyngen/{method}_logcounts_pre.npy', out[0])
# np.save(f'../plots/dyngen/{method}_protein_pre.npy', out[1])

# scScope

In [12]:
# Apply autoencoder
import scscope as DeepImpute
method = 'scScope'
def apply_red(x):
    model = DeepImpute.train(
        x[mask], 32,
        use_mask=True, batch_size=64, max_epoch=100, epoch_per_check=100,
        T=2, exp_batch_idx_input=[], encoder_layers=[], decoder_layers=[],
        learning_rate=0.0001, beta1=0.05)
    latent, output, pred_be = DeepImpute.predict(x, model, batch_effect=[])
    return output
np.save(f'../plots/dyngen/{method}_logcounts.npy', apply_red(d1))
np.save(f'../plots/dyngen/{method}_protein.npy', apply_red(d2))
np.save(f'../plots/dyngen/{method}_logcounts_pre.npy', apply_red(m1))
np.save(f'../plots/dyngen/{method}_protein_pre.npy', apply_red(m2))

Finisheded epoch：100
Current reconstruction error is: 0.3172206815637394
Finish training 1193 samples after 100 epochs. The total training time is 19.26297879219055 seconds.
Finisheded epoch：100
Current reconstruction error is: 0.43602986360500134
Finish training 1193 samples after 100 epochs. The total training time is 19.283692836761475 seconds.
Finisheded epoch：100
Current reconstruction error is: 0.84022135
Finish training 1193 samples after 100 epochs. The total training time is 6.928863525390625 seconds.
Finisheded epoch：100
Current reconstruction error is: 0.15716279
Finish training 1193 samples after 100 epochs. The total training time is 6.928745746612549 seconds.
